In [1]:
from tensorflow.keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Input, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.layers import Conv2D
from keras import backend as k
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import pickle
import tensorflow as tf

In [2]:
def loadPickleData(file):
    data = pickle.load(open(file, 'rb'));
    images = data[0];
    labels = data[1];
    return [images, labels]

In [3]:
[images,labels] = loadPickleData("./../Data/pickledFiles/Num_SingleDigit_3channel.p")

In [4]:
y = np_utils.to_categorical(labels);
print(y);
print(y.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
(2000, 10)


In [5]:
X = images
X.shape
# X = np.reshape(X, (2000,60,40,1))

(2000, 60, 40, 3)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2);

In [7]:
model = applications.VGG19(weights = "imagenet", include_top=False,\
                           input_shape = (60, 40, 3))

In [8]:
print(len(model.layers))
for layer in model.layers[0:19]:
    layer.trainable = False

22


In [9]:
inputt = Input(shape =  (60,40,3), name = 'captcha_input')
output_vgg16_conv = model(inputt);

In [10]:
x = Flatten(name='flatten')(output_vgg16_conv)


In [11]:
predictions = Dense(10, activation="softmax")(x)


In [12]:
model_final = Model(inputt,predictions)


In [13]:
model_final.compile(loss = "categorical_crossentropy",\
                    optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9), metrics=["accuracy"])

In [14]:
print(model_final.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
captcha_input (InputLayer)   [(None, 60, 40, 3)]       0         
_________________________________________________________________
vgg19 (Functional)           (None, 1, 1, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                5130      
Total params: 20,029,514
Trainable params: 4,724,746
Non-trainable params: 15,304,768
_________________________________________________________________
None


In [15]:
lossCheck = ModelCheckpoint('singleCharLoss4.h5', monitor='val_loss', verbose=True, save_best_only=True)
accCheck = ModelCheckpoint('singleCharAcc4.h5', monitor='val_accuracy', verbose=True, save_best_only=True)

In [16]:
history = model_final.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=300, epochs=150, verbose=1, callbacks=[lossCheck, accCheck])

Epoch 1/150
6/6 [==============================] - 9s 1s/step - loss: 11.8875 - accuracy: 0.1312 - val_loss: 4.9015 - val_accuracy: 0.1475

Epoch 00001: val_loss improved from inf to 4.90153, saving model to singleCharLoss4.h5

Epoch 00001: val_accuracy improved from -inf to 0.14750, saving model to singleCharAcc4.h5
Epoch 2/150
6/6 [==============================] - 8s 1s/step - loss: 3.5432 - accuracy: 0.1462 - val_loss: 2.4196 - val_accuracy: 0.1350

Epoch 00002: val_loss improved from 4.90153 to 2.41963, saving model to singleCharLoss4.h5

Epoch 00002: val_accuracy did not improve from 0.14750
Epoch 3/150
6/6 [==============================] - 9s 1s/step - loss: 2.3402 - accuracy: 0.1181 - val_loss: 2.3008 - val_accuracy: 0.1675

Epoch 00003: val_loss improved from 2.41963 to 2.30077, saving model to singleCharLoss4.h5

Epoch 00003: val_accuracy improved from 0.14750 to 0.16750, saving model to singleCharAcc4.h5
Epoch 4/150
6/6 [==============================] - 8s 1s/step - loss: 

6/6 [==============================] - 9s 1s/step - loss: 1.8248 - accuracy: 0.3381 - val_loss: 2.0941 - val_accuracy: 0.2575

Epoch 00030: val_loss improved from 2.12302 to 2.09408, saving model to singleCharLoss4.h5

Epoch 00030: val_accuracy improved from 0.24500 to 0.25750, saving model to singleCharAcc4.h5
Epoch 31/150
6/6 [==============================] - 9s 2s/step - loss: 1.8093 - accuracy: 0.3487 - val_loss: 2.1123 - val_accuracy: 0.2550

Epoch 00031: val_loss did not improve from 2.09408

Epoch 00031: val_accuracy did not improve from 0.25750
Epoch 32/150
6/6 [==============================] - 9s 2s/step - loss: 1.7976 - accuracy: 0.3369 - val_loss: 2.0915 - val_accuracy: 0.2650

Epoch 00032: val_loss improved from 2.09408 to 2.09147, saving model to singleCharLoss4.h5

Epoch 00032: val_accuracy improved from 0.25750 to 0.26500, saving model to singleCharAcc4.h5
Epoch 33/150
6/6 [==============================] - 9s 1s/step - loss: 1.7664 - accuracy: 0.3550 - val_loss: 2.067


Epoch 00058: val_loss did not improve from 1.88148

Epoch 00058: val_accuracy improved from 0.33500 to 0.34000, saving model to singleCharAcc4.h5
Epoch 59/150
6/6 [==============================] - 9s 2s/step - loss: 1.3507 - accuracy: 0.4894 - val_loss: 1.8904 - val_accuracy: 0.3350

Epoch 00059: val_loss did not improve from 1.88148

Epoch 00059: val_accuracy did not improve from 0.34000
Epoch 60/150
6/6 [==============================] - 9s 1s/step - loss: 1.3373 - accuracy: 0.4831 - val_loss: 1.9058 - val_accuracy: 0.3325

Epoch 00060: val_loss did not improve from 1.88148

Epoch 00060: val_accuracy did not improve from 0.34000
Epoch 61/150
6/6 [==============================] - 9s 2s/step - loss: 1.3292 - accuracy: 0.5069 - val_loss: 1.8926 - val_accuracy: 0.3325

Epoch 00061: val_loss did not improve from 1.88148

Epoch 00061: val_accuracy did not improve from 0.34000
Epoch 62/150
6/6 [==============================] - 9s 1s/step - loss: 1.3096 - accuracy: 0.5113 - val_loss: 1.8

6/6 [==============================] - 9s 1s/step - loss: 1.0250 - accuracy: 0.6250 - val_loss: 1.9302 - val_accuracy: 0.3775

Epoch 00090: val_loss did not improve from 1.86865

Epoch 00090: val_accuracy did not improve from 0.39250
Epoch 91/150
6/6 [==============================] - 9s 1s/step - loss: 1.0186 - accuracy: 0.6244 - val_loss: 1.9154 - val_accuracy: 0.3850

Epoch 00091: val_loss did not improve from 1.86865

Epoch 00091: val_accuracy did not improve from 0.39250
Epoch 92/150
6/6 [==============================] - 9s 1s/step - loss: 1.0087 - accuracy: 0.6331 - val_loss: 1.8670 - val_accuracy: 0.3775

Epoch 00092: val_loss improved from 1.86865 to 1.86700, saving model to singleCharLoss4.h5

Epoch 00092: val_accuracy did not improve from 0.39250
Epoch 93/150
6/6 [==============================] - 14s 2s/step - loss: 1.0044 - accuracy: 0.6344 - val_loss: 1.9103 - val_accuracy: 0.3750

Epoch 00093: val_loss did not improve from 1.86700

Epoch 00093: val_accuracy did not impro

6/6 [==============================] - 14s 2s/step - loss: 0.7767 - accuracy: 0.7306 - val_loss: 1.9579 - val_accuracy: 0.4050

Epoch 00122: val_loss did not improve from 1.86700

Epoch 00122: val_accuracy did not improve from 0.42000
Epoch 123/150
6/6 [==============================] - 10s 2s/step - loss: 0.7837 - accuracy: 0.7312 - val_loss: 1.9468 - val_accuracy: 0.4175

Epoch 00123: val_loss did not improve from 1.86700

Epoch 00123: val_accuracy did not improve from 0.42000
Epoch 124/150
6/6 [==============================] - 9s 1s/step - loss: 0.7626 - accuracy: 0.7350 - val_loss: 1.9697 - val_accuracy: 0.4100

Epoch 00124: val_loss did not improve from 1.86700

Epoch 00124: val_accuracy did not improve from 0.42000
Epoch 125/150
6/6 [==============================] - 9s 1s/step - loss: 0.7639 - accuracy: 0.7444 - val_loss: 1.9842 - val_accuracy: 0.4100

Epoch 00125: val_loss did not improve from 1.86700

Epoch 00125: val_accuracy did not improve from 0.42000
Epoch 126/150
6/6 [=

In [17]:
history1 = model_final.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=300, epochs=150, verbose=1, callbacks=[lossCheck, accCheck])

Epoch 1/150
6/6 [==============================] - 8s 1s/step - loss: 0.6203 - accuracy: 0.7906 - val_loss: 2.0696 - val_accuracy: 0.4100

Epoch 00001: val_loss did not improve from 1.86700

Epoch 00001: val_accuracy did not improve from 0.43000
Epoch 2/150
6/6 [==============================] - 8s 1s/step - loss: 0.6173 - accuracy: 0.7937 - val_loss: 2.0392 - val_accuracy: 0.4275

Epoch 00002: val_loss did not improve from 1.86700

Epoch 00002: val_accuracy did not improve from 0.43000
Epoch 3/150
6/6 [==============================] - 8s 1s/step - loss: 0.6108 - accuracy: 0.7906 - val_loss: 2.0537 - val_accuracy: 0.4225

Epoch 00003: val_loss did not improve from 1.86700

Epoch 00003: val_accuracy did not improve from 0.43000
Epoch 4/150
6/6 [==============================] - 8s 1s/step - loss: 0.6037 - accuracy: 0.7994 - val_loss: 2.0746 - val_accuracy: 0.4200

Epoch 00004: val_loss did not improve from 1.86700

Epoch 00004: val_accuracy did not improve from 0.43000
Epoch 5/150
6/6 

6/6 [==============================] - 9s 1s/step - loss: 0.4861 - accuracy: 0.8481 - val_loss: 2.1713 - val_accuracy: 0.4250

Epoch 00034: val_loss did not improve from 1.86700

Epoch 00034: val_accuracy did not improve from 0.43250
Epoch 35/150
6/6 [==============================] - 9s 1s/step - loss: 0.4872 - accuracy: 0.8431 - val_loss: 2.1680 - val_accuracy: 0.4350

Epoch 00035: val_loss did not improve from 1.86700

Epoch 00035: val_accuracy improved from 0.43250 to 0.43500, saving model to singleCharAcc4.h5
Epoch 36/150
6/6 [==============================] - 9s 1s/step - loss: 0.4756 - accuracy: 0.8481 - val_loss: 2.1689 - val_accuracy: 0.4300

Epoch 00036: val_loss did not improve from 1.86700

Epoch 00036: val_accuracy did not improve from 0.43500
Epoch 37/150
6/6 [==============================] - 9s 1s/step - loss: 0.4703 - accuracy: 0.8481 - val_loss: 2.2124 - val_accuracy: 0.4225

Epoch 00037: val_loss did not improve from 1.86700

Epoch 00037: val_accuracy did not improve

Epoch 67/150
6/6 [==============================] - 9s 2s/step - loss: 0.3779 - accuracy: 0.8850 - val_loss: 2.3438 - val_accuracy: 0.4275

Epoch 00067: val_loss did not improve from 1.86700

Epoch 00067: val_accuracy did not improve from 0.44750
Epoch 68/150
6/6 [==============================] - 9s 2s/step - loss: 0.3809 - accuracy: 0.8781 - val_loss: 2.3389 - val_accuracy: 0.4375

Epoch 00068: val_loss did not improve from 1.86700

Epoch 00068: val_accuracy did not improve from 0.44750
Epoch 69/150
6/6 [==============================] - 9s 2s/step - loss: 0.3755 - accuracy: 0.8844 - val_loss: 2.3620 - val_accuracy: 0.4350

Epoch 00069: val_loss did not improve from 1.86700

Epoch 00069: val_accuracy did not improve from 0.44750
Epoch 70/150
6/6 [==============================] - 9s 2s/step - loss: 0.3719 - accuracy: 0.8856 - val_loss: 2.3107 - val_accuracy: 0.4350

Epoch 00070: val_loss did not improve from 1.86700

Epoch 00070: val_accuracy did not improve from 0.44750
Epoch 71/150

6/6 [==============================] - 9s 2s/step - loss: 0.2784 - accuracy: 0.9206 - val_loss: 2.5117 - val_accuracy: 0.4450

Epoch 00109: val_loss did not improve from 1.86700

Epoch 00109: val_accuracy did not improve from 0.45500
Epoch 110/150
6/6 [==============================] - 9s 2s/step - loss: 0.2761 - accuracy: 0.9200 - val_loss: 2.5115 - val_accuracy: 0.4475

Epoch 00110: val_loss did not improve from 1.86700

Epoch 00110: val_accuracy did not improve from 0.45500
Epoch 111/150
6/6 [==============================] - 9s 2s/step - loss: 0.2759 - accuracy: 0.9181 - val_loss: 2.4822 - val_accuracy: 0.4425

Epoch 00111: val_loss did not improve from 1.86700

Epoch 00111: val_accuracy did not improve from 0.45500
Epoch 112/150
6/6 [==============================] - 9s 2s/step - loss: 0.2745 - accuracy: 0.9219 - val_loss: 2.5288 - val_accuracy: 0.4450

Epoch 00112: val_loss did not improve from 1.86700

Epoch 00112: val_accuracy did not improve from 0.45500
Epoch 113/150
6/6 [===

6/6 [==============================] - 9s 2s/step - loss: 0.2234 - accuracy: 0.9325 - val_loss: 2.6329 - val_accuracy: 0.4350

Epoch 00142: val_loss did not improve from 1.86700

Epoch 00142: val_accuracy did not improve from 0.46000
Epoch 143/150
6/6 [==============================] - 9s 2s/step - loss: 0.2218 - accuracy: 0.9331 - val_loss: 2.6460 - val_accuracy: 0.4400

Epoch 00143: val_loss did not improve from 1.86700

Epoch 00143: val_accuracy did not improve from 0.46000
Epoch 144/150
6/6 [==============================] - 9s 2s/step - loss: 0.2214 - accuracy: 0.9306 - val_loss: 2.6620 - val_accuracy: 0.4450

Epoch 00144: val_loss did not improve from 1.86700

Epoch 00144: val_accuracy did not improve from 0.46000
Epoch 145/150
6/6 [==============================] - 9s 2s/step - loss: 0.2202 - accuracy: 0.9350 - val_loss: 2.6901 - val_accuracy: 0.4475

Epoch 00145: val_loss did not improve from 1.86700

Epoch 00145: val_accuracy did not improve from 0.46000
Epoch 146/150
6/6 [===

In [57]:
history2 = model_final.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=300, epochs=50, verbose=1);

Epoch 1/50
6/6 [==============================] - 8s 1s/step - loss: 0.7408 - accuracy: 0.7550 - val_loss: 1.7918 - val_accuracy: 0.4675
Epoch 2/50
6/6 [==============================] - 8s 1s/step - loss: 0.7422 - accuracy: 0.7575 - val_loss: 1.8043 - val_accuracy: 0.4625
Epoch 3/50
6/6 [==============================] - 8s 1s/step - loss: 0.7343 - accuracy: 0.7569 - val_loss: 1.8027 - val_accuracy: 0.4650
Epoch 4/50
6/6 [==============================] - 8s 1s/step - loss: 0.7151 - accuracy: 0.7619 - val_loss: 1.7949 - val_accuracy: 0.4775
Epoch 5/50
6/6 [==============================] - 8s 1s/step - loss: 0.7033 - accuracy: 0.7713 - val_loss: 1.8078 - val_accuracy: 0.4525
Epoch 6/50
6/6 [==============================] - 8s 1s/step - loss: 0.6999 - accuracy: 0.7706 - val_loss: 1.7969 - val_accuracy: 0.4775
Epoch 7/50
6/6 [==============================] - 8s 1s/step - loss: 0.6904 - accuracy: 0.7756 - val_loss: 1.8212 - val_accuracy: 0.4575
Epoch 8/50
6/6 [=========================

In [18]:
max(max(history.history["val_accuracy"]), max(history1.history["val_accuracy"]))

0.46000000834465027

In [63]:
xtr = model_final.predict(X_train, batch_size = 200, verbose=1)

8/8 [==============================] - 5s 569ms/step


In [64]:
Xtest = model_final.predict(X_test, batch_size = 100, verbose=1);

4/4 [==============================] - 1s 294ms/step


In [65]:
print(xtr.shape)

(1600, 10)


In [66]:
resize = np.prod(xtr.shape[1:])

In [67]:
xtr = np.reshape(xtr, (1600, resize));
Xtest = np.reshape(Xtest, (400, resize));

In [74]:
bf_train_s = xtr
bf_test_s = Xtest

In [75]:
y_train_mc = np.argmax(y_train, axis=1)
y_test_mc = np.argmax(y_test, axis = 1)

In [80]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm


In [77]:
clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', verbose=1,max_iter=10000)

In [78]:
clf.fit(bf_train_s,y_train_mc)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


LogisticRegression(max_iter=10000, multi_class='multinomial', verbose=1)

In [79]:
print(clf.score(bf_test_s, y_test_mc))

0.4675


In [81]:
multiClassSVM = svm.LinearSVC(loss = 'squared_hinge', penalty = 'l2',C=1/5000 ,max_iter=800000,verbose=1) 
multiClassSVM.fit(bf_train_s,y_train_mc)

[LibLinear]

LinearSVC(C=0.0002, max_iter=800000, verbose=1)

In [82]:
multiClassSVM.score(bf_test_s, y_test_mc)

0.465